# PROBLEMA 5 LISTA 2

### CARGAR LOS DATOS

In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

try:
    dataset = fetch_ucirepo(id=544)
    data = pd.DataFrame(dataset.data['features'])
    print(dataset)

    target = pd.DataFrame(dataset.data['targets'], columns=["NObeyesdad"])

    full_data = pd.concat([data, target], axis=1)

except Exception as ex:
    print(ex)


{'data': {'ids': None, 'features':       Gender        Age    Height      Weight family_history_with_overweight  \
0     Female  21.000000  1.620000   64.000000                            yes   
1     Female  21.000000  1.520000   56.000000                            yes   
2       Male  23.000000  1.800000   77.000000                            yes   
3       Male  27.000000  1.800000   87.000000                             no   
4       Male  22.000000  1.780000   89.800000                             no   
...      ...        ...       ...         ...                            ...   
2106  Female  20.976842  1.710730  131.408528                            yes   
2107  Female  21.982942  1.748584  133.742943                            yes   
2108  Female  22.524036  1.752206  133.689352                            yes   
2109  Female  24.361936  1.739450  133.346641                            yes   
2110  Female  23.664709  1.738836  133.472641                            yes   

    

### VISUALIZACION DE LOS DATOS

In [2]:
categorical_columns = list(full_data.dtypes[full_data.dtypes == 'O'].index.values)
print("Variables Categoricas:", categorical_columns)

print(full_data.columns)


# Obtener el tipo de cada variable
tipos_variables = full_data.dtypes

# Contar valores únicos por cada columna
valores_unicos = full_data.nunique()

# Combinar ambos resultados en un solo DataFrame para una vista más clara
resumen = pd.DataFrame({'Tipo': tipos_variables, 'Valores Únicos': valores_unicos})
print(resumen)

print(full_data['CAEC'].unique())

full_data.head()

Variables Categoricas:

 ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']
Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')
                                   Tipo  Valores Únicos
Gender                           object               2
Age                             float64            1402
Height                          float64            1574
Weight                          float64            1525
family_history_with_overweight   object               2
FAVC                             object               2
FCVC                            float64             810
NCP                             float64             635
CAEC                             object               4
SMOKE                            object               2
CH2O                            float64            1268
SC

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
